https://adventofcode.com/2022/day/19

In [1]:
import numpy as np
from dataclasses import dataclass, field
from tqdm.notebook import tqdm
import re

In [2]:
@dataclass
class RobotFactory():
    blueprint: int
    costs: np.array
    robots: np.array=field(init=False)
    resources: np.array=field(init=False)
        
    def __post_init__(self):
        self.robots = np.array([1,0,0,0])
        self.resources = np.array([0,0,0,0])
    
    def quality_level(self, time):
        return self.n_geodes(time) * self.blueprint
    
    def n_geodes(self, time):
        return list(V_state(self.robots, self.resources, self.costs, T=1, T_max=time))[0]
    
    def __hash__(self):
        return hash(tuple(self.robots) + tuple(self.resources))
    
    @classmethod
    def from_input(cls, n):
        return RobotFactory(blueprint=n[0],
                            costs=np.array([[n[1], 0, 0, 0],
                                            [n[2], 0, 0, 0],
                                            [n[3], n[4], 0, 0],
                                            [n[5], 0, n[6], 0]]))

In [3]:
def can_build(costs, resources):
    return np.all((costs - resources) <= 0, axis=1)
    
def useful_to_build(costs, robots, i):
    return np.any(np.where(costs - robots > 0, costs-robots, 0)[:,i])

# Backward induction
def V_state(robots, resources, costs, T, T_max, best_robot=0, did_not_build=(), seen=None):
    if seen is None:
        seen = {}
    V = robots[3]
    if T == T_max:
        value = V
        policies = [-1]
        return {value: policies}
    else:
        can_build_now = can_build(costs, resources)
        next_policies = V_state(robots,
                                resources+robots,
                                costs,
                                T+1,
                                T_max=T_max,
                                best_robot=best_robot,
                                did_not_build=can_build_now,
                                seen=seen)
        p = max(next_policies.keys())
        values = [V + p]
        state = tuple(robots) + tuple(resources)
        if state not in seen:
            seen[state] = T
        else:
            seen[state] = min(seen[state], T)
        policies = [[-1] + next_policies[p]]
        options = (i for i in [0, 1, 2, 3]
                   if (i >= best_robot-1) and 
                      (i not in list(np.where(did_not_build)[0])))
        for policy in options:
            if (not useful_to_build(costs, robots, policy)) and (policy != 3): continue
            if can_build_now[policy]:
                next_resources = resources - costs[policy] + robots
                next_robots = robots + np.eye(4, dtype=int)[policy]
                state = tuple(next_robots) + tuple(next_resources)
                if T_old := seen.get(state):
                    if T_old <= T: continue                 
                next_policies = V_state(next_robots,
                                        next_resources,
                                        costs,
                                        T+1,
                                        T_max=T_max,
                                        best_robot=policy,
                                        seen=seen)
                p = max(next_policies.keys())
                values.append(V+p)
                if state not in seen:
                    seen[state] = T
                else:
                    seen[state] = min(seen[state], T)
                
                policies.append([policy] + next_policies[p])
    possible_options = {v: p for v, p in zip(values, policies)}
    return possible_options
    
def read_data(filename):
    with open(filename, 'r') as f:
        return f.readlines()

def solve_part_1(filename):
    summed_quality_level = 0
    for line in tqdm(read_data(filename)):
        n = list(map(int, re.findall('\d+', line)))
        rf = RobotFactory.from_input(n) 
        summed_quality_level += rf.quality_level(time=24)
    return summed_quality_level

def solve_part_2(filename):
    geode_product = 1
    for line in tqdm(read_data(filename)[:3]):
        n = list(map(int, re.findall('\d+', line)))
        rf = RobotFactory.from_input(n)   
        geode_product *= rf.n_geodes(time=32)
    return geode_product   

In [4]:
filename = "../example_data/day19_example_data.txt"
solve_part_1(filename)

  0%|          | 0/2 [00:00<?, ?it/s]

33

In [5]:
filename = "../data/day19_data.txt"
solve_part_1(filename)

  0%|          | 0/30 [00:00<?, ?it/s]

1262

In [6]:
filename = "../data/day19_data.txt"
solve_part_2(filename)

  0%|          | 0/3 [00:00<?, ?it/s]

37191